In [1]:
!zenml stack set default

Active repository stack set to: 'default'.
⠙ Setting the repository active stack to 'default'...t'...


In [2]:
from zenml import pipeline, step

@step
def my_step():
    print("Hello world!")

@pipeline
def my_pipeline():
    my_step()

my_pipeline()


Initiating a new run for the pipeline: my_pipeline.
Executing a new run.
Using user: alex.ext@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Dashboard URL for Pipeline Run: https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/8a462fb6-b1fe-48df-9677-edc76bc8352d/runs/5ce32934-0a24-4121-a396-e46b053b6fce
Using cached version of my_step.
Step my_step has started.
Pipeline run has finished in 1.754s.


PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2024, 9, 11, 19, 7, 52), updated=datetime.datetime(2024, 9, 11, 19, 7, 54), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 2, 8, 8, 8, 21), updated=datetime.datetime(2024, 9, 11, 18, 54, 43), active=True, activation_token=None, full_name='Alex Strick van Linschoten', email_opted_in=True, is_service_account=False, is_admin=False), metadata=None, resources=None, id=UUID('f4982973-1f26-41d3-a3a1-e2000f5427cb'), permission_denied=False, name='alex.ext@zenml.io'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2023, 11, 30, 9, 39, 29), updated=datetime.datetime(2023, 11, 30, 9, 39, 29), user=None), metadata=None, resources=None, id=UUID('a4cd4161-6ee5-411c-8adf-559ac084ceb5'), permission_denied=False, name='default'), pipeline=PipelineResponse(body=PipelineResponseBody(created=datetime.datetime(2024, 6, 13, 16, 10, 35), up

In [3]:
@step
def some_other_step():
    print("This is another step!")

some_other_step()

Running single step pipeline to execute step some_other_step
Initiating a new run for the pipeline: some_other_step.
Executing a new run.
Caching is disabled by default for some_other_step.
Using user: alex.ext@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Dashboard URL for Pipeline Run: https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/8a462fb6-b1fe-48df-9677-edc76bc8352d/runs/9847438a-7ee2-44f0-a1a1-a2fc2d90f101
Step some_other_step has started.
This is another step!
Step some_other_step has finished in 0.740s.
Step some_other_step completed successfully.
Pipeline run has finished in 1.804s.


In [4]:
!zenml stack set az_k8s_pigeon

Active repository stack set to: 'az_k8s_pigeon'.
⠙ Setting the repository active stack to 'az_k8s_pigeon'...n'...


In [1]:
from zenml.integrations.kubernetes.flavors import (
    KubernetesOrchestratorSettings,
)

MNT_PATH = "/mnt/data"

kubernetes_settings = KubernetesOrchestratorSettings(
    pod_settings={
        "affinity": {
            "nodeAffinity": {
                "requiredDuringSchedulingIgnoredDuringExecution": {
                    "nodeSelectorTerms": [
                        {
                            "matchExpressions": [
                                {
                                    "key": "zenml.io/gpu",
                                    "operator": "In",
                                    "values": ["yes"],
                                }
                            ]
                        }
                    ]
                }
            }
        },
        "volumes": [
            {
                "name": "data-volume",
                "persistentVolumeClaim": {"claimName": "pvc-managed-premium"},
            }
        ],
        "volume_mounts": [{"name": "data-volume", "mountPath": MNT_PATH}],
    },
)



In [2]:
import os
from zenml.config import DockerSettings

docker_settings = DockerSettings(
    parent_image="pytorch/pytorch:2.2.2-cuda11.8-cudnn8-runtime",
    environment={
        "PJRT_DEVICE": "CUDA",
        "USE_TORCH_XLA": "false",
        "MKL_SERVICE_FORCE_INTEL": 1,
        "HF_TOKEN": os.environ["HF_TOKEN"],
        "HF_HOME": MNT_PATH,
    },
    python_package_installer="uv",
    requirements="requirements.txt",
    python_package_installer_args={
        "system": None,
    },
    apt_packages=["git", "ffmpeg"],
    # prevent_build_reuse=True,
)

In [4]:
from zenml import pipeline, step

@step(settings={"orchestrator.kubernetes": kubernetes_settings})
def k8s_step():
    print("Training model...")

@pipeline(settings={"docker": docker_settings})
def k8s_pipeline():
    k8s_step()

k8s_pipeline()

Initiating a new run for the pipeline: k8s_pipeline.
Uploading notebook code...
Upload finished.
Unable to find a build to reuse. A previous build can be reused when the following conditions are met:
  * The existing build was created for the same stack, ZenML version and Python version
  * The stack contains a container registry
  * The Docker settings of the pipeline and all its steps are the same as for the existing build.
Building Docker image(s) for pipeline k8s_pipeline.
Building Docker image demozenmlcontainerregistry.azurecr.io/zenml:k8s_pipeline-orchestrator.
- Including stack requirements: adlfs>=2021.10.0, azure-ai-ml==1.18.0, azure-identity, azure-keyvault-keys, azure-keyvault-secrets, azure-mgmt-containerservice>=20.0.0, azure-storage-blob==12.17.0, azureml-core==1.56.0, ipywidgets>=8.0.0, kubernetes, kubernetes>=21.7,<26
- Including user-defined requirements from file /Users/strickvl/coding/zenml/repos/zenml-projects/mlops-community-demo/requirements.txt
Including apt pac